In [129]:
import pandas as pd
import numpy as np
import preprocessingFuncts as pp
from sklearn import neighbors
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt

In [130]:
#all base datas

def readData(paths, base, categories=[]):
  arr = []
  itemData = pp.readItemData()
  userData = pp.readUserData()
  for path in paths:
    ratingData = pp.readRatingData(path)
    data = 0
    if base == "user":
      data = pp.specifyByUserData(userData, ratingData, categories)
    elif base == "item":
      data = pp.specifyByItemData(itemData, ratingData, categories)
    else:
      raise Exception(
          "readData(path, base, categories) base can only be \"user\" or \"item\"")
    data = data.dropna()
    y = data.loc[:, 'rating']
    x = data.copy()
    x = x.drop(['rating'], axis='columns')
    arr.append({'x': x, 'y': y, 'name': path[8:10], 'type': path[-4:]})
  return arr

In [131]:
def makeUserItemMatrix(path):
  ratingData = pp.readRatingData(path)
  UI = ratingData.pivot_table(index='user_id',columns='item_id',values='rating')
  UI = UI.set_axis([int(x) for x in UI.columns], axis='columns', inplace=False)
  UI = UI.set_axis([int(x) for x in UI.index], axis='index', inplace=False)
  return UI

In [132]:
#Got the matrix
matrix = makeUserItemMatrix('ml-100k\\ua.base')
matrix

,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
import sklearn.metrics as metrics
import scipy.spatial.distance as sp_dist

#getting the similarity of the movie with each 

def similarityItem(matrix,itemId,method='pearson'):
  temp = matrix.corrwith(matrix[itemId], method=method)
  temp = pd.DataFrame(temp, columns=['corr']).reset_index()
  timesRated = pd.DataFrame(matrix.count(), columns=['times_rated']).reset_index()
  temp = temp.merge(timesRated)
  temp = temp.rename({'index':'item_id'},axis='columns')
  temp = temp.dropna()
  temp = temp[temp['item_id'] != itemId]
  # TODO - change that 30 to a threshold variable
  return temp[temp['times_rated'] >= 30].sort_values(by='corr', ascending=False)

In [134]:
kNeighbours = 5 # can set this number however you wish for the number of neighbours
# TODO - make this shit a class for better accessibility

def userRatedMovies(userId,matrix):
  pass

def predict(userId,itemId,k,matrix):
  a,b = 0,0
  itemSim = similarityItem(matrix, itemId)
  similarItemsIds = itemSim['item_id'].values
  similarities = itemSim['corr'].values
  # print(similarItemsIds)
  # print(similarities)
  _k = k
  _idx = 0
  while _k>0:
    if(not pd.isna(matrix.iloc[userId, similarItemsIds[_idx]])):
      a += similarities[_idx]*matrix.iloc[userId, similarItemsIds[_idx]]
      b += similarities[_idx]
      _k -=1
    _idx += 1
  return round(a/b)

# TODO - turn this to it's own seperate file and also do cross eval, percision, recall, accuracy, etc..
predict(3,331,kNeighbours,matrix)


c:\Users\scarl\anaconda3\lib\site-packages\numpy\lib\function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
c:\Users\scarl\anaconda3\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


4